<a href="https://colab.research.google.com/github/techwithanirudh/Automatic-Image-Captioning/blob/master/Capture_Img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask_ngrok flask_cors

In [ ]:
%%html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>
<body>
    <div class="form-group">  
        <div class="camera">
            <video id="video">Video stream not available.</video>
            <button type="button" id="startbutton" class="btn btn-success" data-toggle="modal" data- 
            target="#modal-success">Take photo</button>
        </div>

        <canvas id="canvas"></canvas>        
        <script>
            (function() {
                // The width and height of the captured photo. We will set the
                // width to the value defined here, but the height will be
                // calculated based on the aspect ratio of the input stream.
            
                var width = 770;    // We will scale the photo width to this
                var height = 900;   // This will be computed based on the input stream
            
                // |streaming| indicates whether or not we're currently streaming
                // video from the camera. Obviously, we start at false.
            
                var streaming = false;
            
                // The various HTML elements we need to configure or control. These
                // will be set by the startup() function.
            
                var video = null;
                var canvas = null;
                var photo = null;
                var startbutton = null;
            
                function startup() {
                  video = document.getElementById('video');
                  canvas = document.getElementById('canvas');
                  photo = document.getElementById('photo');
                  startbutton = document.getElementById('startbutton');
                
                  navigator.mediaDevices.getUserMedia({video: true, audio: false})
                  .then(function(stream) {
                    video.srcObject = stream;
                    video.play();
                  })
                  .catch(function(err) {
                    console.log("An error occurred: " + err);
                  });
              
                  video.addEventListener('canplay', function(ev){
                    if (!streaming) {
                      height = video.videoHeight / (video.videoWidth/width);
                    
                      // Firefox currently has a bug where the height can't be read from
                      // the video, so we will make assumptions if this happens.
                    
                      if (isNaN(height)) {
                        height = width / (4/3);
                      }
                  
                      video.setAttribute('width', 340);
                      video.setAttribute('height', 300);
                      canvas.setAttribute('width', width);
                      canvas.setAttribute('height', height);
                      streaming = true;
                    }
                  }, false);
              
                  startbutton.addEventListener('click', function(ev){
                    takepicture();
                    ev.preventDefault();
                  }, false);
              
                  clearphoto();
                }
            
                // Fill the photo with an indication that none has been
                // captured.
            
                function clearphoto() {
                  var context = canvas.getContext('2d');
                  context.fillStyle = "#AAA";
                  context.fillRect(0, 0, canvas.width, canvas.height);
                
                  var data = canvas.toDataURL('image/png');
                }
            
                // Capture a photo by fetching the current contents of the video
                // and drawing it into a canvas, then converting that to a PNG
                // format data URL. By drawing it on an offscreen canvas and then
                // drawing that to the screen, we can change its size and/or apply
                // other changes before drawing it.
            
                function takepicture() {
                  var context = canvas.getContext('2d');
                  if (width && height) {
                    canvas.width = width;
                    canvas.height = height;
                    context.drawImage(video, 0, 0, width, height);
                
                    var data = canvas.toDataURL('image/jpeg');
                    var request = new XMLHttpRequest();
                    request.open("POST", 'SERVER_URL');
                    request.send(data);
                  } else {
                    clearphoto();
                  }
                }
            
                // Set up our event listener to run the startup process
                // once loading is complete.
                window.addEventListener('load', startup, false);
              })(); 
          </script>
</body>
</html>

In [ ]:
from flask import Flask, render_template, jsonify, request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from IPython.display import display

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

@app.route('/', methods=['GET','POST'])
def getimg():
  data = request.data
  display(data.decode("utf-8"))
  return ''

app.run()